In [3]:
#!pip install lxml

import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
#import time
import datetime
import socket

TIMEOUT_SEC = 10 # default timeout in seconds
socket.setdefaulttimeout(TIMEOUT_SEC)

def getAndParseURL(url):
    result = requests.get(url,headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

html = getAndParseURL("http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_01")
html_footer = html.find_all("table", {"class": "tb_base tb_footer"})
bs_text_tag = html_footer[0].select('td')[0]
modification_date_regex = r"Última modificação: (\d{2})/(\d{2})/(\d{2})"
match = re.search(modification_date_regex, bs_text_tag.get_text())

if match:
    day, month, year = match.groups()
    modified_date = datetime.datetime(year=int(year)+2000, month=int(month), day=int(day))

    print("Última modificação:", modified_date.strftime("%d-%m-%Y"))
else:
    print("Não foi possível encontrar a data da 'Última modificação' do site")

Última modificação: 21-12-2023


In [ ]:
base_url = 'http://vitibrasil.cnpuv.embrapa.br/download/'

files = {'file_dot_comma': ['Producao.csv',
        'ProcessaViniferas.csv',
        'Comercio.csv',
        'ImpVinhos.csv',
        'ImpEspumantes.csv',
        'ImpFrescas.csv',
        'ImpPassas.csv',
        'ImpSuco.csv',
        'ExpVinho.csv',
        'ExpEspumantes.csv',
        'ExpUva.csv',
        'ExpSuco.csv'],
        'file_t': ['ProcessaAmericanas.csv',
        'ProcessaMesa.csv'] }

file_sep = {
    'file_dot_comma': ';',
    'file_t': '\t'
}

dataframes = []
for file_type, filenames in files.items():
    for filename in filenames:
        try:
            df = pd.read_csv(base_url + filename, sep=file_sep[file_type])
            dataframes.append(df)
        except Exception as e:
            df = pd.DataFrame()  
            dataframes.append(df)
            print(f"Error reading {filename}: {e}")

df_dict = {
    'Producao': dataframes[0],
    'ProcessaViniferas': dataframes[1],
    'ProcessaAmericanas': dataframes[2],
    'ProcessaMesa': dataframes[3],
    'Comercio': dataframes[4],
    'ImpVinhos': dataframes[5],
    'ImpEspumantes': dataframes[6],
    'ImpFrescas': dataframes[7],
    'ImpPassas': dataframes[8],
    'ImpSuco': dataframes[9],
    'ExpVinho': dataframes[10],
    'ExpEspumantes': dataframes[11],
    'ExpUva': dataframes[12],
    'ExpSuco': dataframes[13]
}

In [2]:
df_dict['Producao']

,id,control,produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,217208604,154264651,146953297,116710345,193875345,177401209,144565438,...,196173123,210308560,86319015,255015187,218375636,144629737,124200414,173899995,195031611,169762429
1,2,vm_Tinto,Tinto,174224052,121133369,118180926,88589019,146544484,144274134,118360170,...,157776363,169811472,75279191,1365957,188270142,121045115,103916391,146075996,162844214,139320884
2,3,vm_Branco,Branco,748400,1160500,1812367,243900,4138768,1441507,1871473,...,37438069,39557250,10727099,217527985,29229970,22032828,19568734,26432799,30198430,27910299
3,4,vm_Rosado,Rosado,42236152,31970782,26960004,27877426,43192093,31685568,24333795,...,958691,939838,312725,36121245,875524,1551794,715289,1391200,1988968,2531246
4,5,VINHO FINO DE MESA (VINIFERA),VINHO FINO DE MESA (VINIFERA),23899346,23586062,21078771,12368410,31644124,39424590,34500590,...,38464314,37148982,18070626,44537870,38707220,37615422,32516686,43474998,47511796,46268556
5,6,vv_Tinto,Tinto,7591557,7265666,6782837,3419625,10047658,14731106,12405154,...,17208996,16745896,8774847,21442212,19118254,17389377,15451883,20433249,24417918,23615783
6,7,vv_Branco,Branco,15562889,15655709,13289304,7930070,18927471,21933695,19874659,...,20054804,19561966,8705066,21928400,18297257,18193055,15487915,20867999,20896613,20693437
7,8,vv_Rosado,Rosado,744900,664687,1006630,1018715,2668995,2759789,2220777,...,1200514,841120,590713,1167258,1291709,2032990,1576888,2173750,2197265,1959336
8,9,SUCO,SUCO,1097771,2296654,3509440,0,0,0,0,...,76723537,87894468,42210389,77161971,65467906,77805352,69261287,100932264,65809079,67045238
9,10,su_Suco de uva simples,Suco de uva integral,1097771,2296654,3509440,0,0,0,0,...,43331223,52233155,31117869,46865626,34367996,50239767,40718523,68038479,35248305,38122173
